## Omics Analysis with Apache Spark
### CADD編

今回はCADDのデータをApache Sparkのデータフレームへ取り込んでいきます。  


### データの下準備
予め以下のデータをダウンロードしておいてあります。 
annotationがincludeされるかどうかでサイズにかなりの開きがあります。
```
~ # ls /zfs-archive/CADD/v1.6/GRCh38/ -sh
合計 1010G
8.5K GRCh38_v1.6                        7.2G gnomad.genomes.r3.0.indel_inclAnno.tsv.bgz   81G whole_genome_SNVs.tsv.gz
206G annotationsGRCh38_v1.6.tar.gz      7.2G gnomad.genomes.r3.0.indel_inclAnno.tsv.gz   308G whole_genome_SNVs_inclAnno.tsv.bgz
1.1G gnomad.genomes.r3.0.indel.tsv.bgz  6.0G gnomad.genomes.r3.0.snv.tsv.gz              314G whole_genome_SNVs_inclAnno.tsv.gz
1.1G gnomad.genomes.r3.0.indel.tsv.gz    81G whole_genome_SNVs.tsv.bgz
```
bgzファイルはApache Sparkで高速に並列アクセスをするために事前につくっておきました。
```
~ # cat gnomad.genomes.r3.0.indel.tsv.gz |gzip -d |sed '1d' |bgzip -c > gnomad.genomes.r3.0.indel.tsv.bgz
```

sed '1d'はheaderを整えるために付与しています。
- 変更前
```
# cat gnomad.genomes.r3.0.indel.tsv.gz |gunzip|head
## CADD GRCh38-v1.6 (c) University of Washington, Hudson-Alpha Institute for Biotechnology and Berlin Institute of Health 2013-2020. All rights reserved.
#Chrom	Pos	Ref	Alt	RawScore	PHRED
1	10061	T	TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC	0.199964	3.123
1	10067	T	TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC	0.225348	3.416
1	10108	C	CA	0.390514	5.321
1	10108	C	CAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT	0.215572	3.303
1	10108	CAACCCT	C	0.317701	4.494
1	10109	AACCCT	A	0.321485	4.538
1	10113	CT	C	0.337989	4.727
1	10114	TA	T	0.338849	4.737
```

- 変更後
```
/zfs-archive/CADD/v1.6/GRCh38 # cat gnomad.genomes.r3.0.indel.tsv.bgz |gunzip|head
#Chrom	Pos	Ref	Alt	RawScore	PHRED
1	10061	T	TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC	0.199964	3.123
1	10067	T	TAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC	0.225348	3.416
1	10108	C	CA	0.390514	5.321
1	10108	C	CAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT	0.215572	3.303
1	10108	CAACCCT	C	0.317701	4.494
1	10109	AACCCT	A	0.321485	4.538
1	10113	CT	C	0.337989	4.727
1	10114	TA	T	0.338849	4.737
1	10116	AC	A	0.337997	4.727
```

データはs3互換ストレージに置いておきます。  
(今回はオンプレミスに構築してあるテストマシンにて実行。s3はminioで、sparkはyarnで動作させています。)

### 起動
あらかじめ設定しておいたAWSの"Service Catalog"からパラメータを入力し、欲しいサイズのSpark Clusterを起動していきます。  



### データの読み込み
今回はgnomad.genomes.r3.0.indel_inclAnno.tsv.bgzを読み込んでみます。  
初めて読み込むデータでもあるため、まずはinferSchema=Trueで読み込んでみます。  


In [ ]:
delta_output_path = 's3a://path-to-data/gnomad.genomes.r3.0.indel_inclAnno'

In [ ]:
df = spark.read.option("delimiter","\t")\
    .csv("s3a://path-to-data/CADD/v1.6/GRCh38/gnomad.genomes.r3.0.indel_inclAnno.tsv.bgz",inferSchema=True,nullValue="NA",header=True)

In [ ]:
import pandas as pd

いくつかの項目を選択し表示させてみます

In [ ]:
display(df.select("#Chrom","Pos","Ref","Alt","Type").limit(10).toPandas())

データの中身を確認してみます。  
df.na.drop()はデータに欠損値があるものをdropするものですが、このデータにおいては何かしらの列でNAがあるようで、何も残りませんでした。  
以下いくつかのセルを実行することでどのように欠損値を含んだデータであるかが確認できますが、表示の都合で出力を削除してあります。

In [ ]:
display(df.na.drop().limit(50).toPandas())

SIFTcatはすべてNAのようです。

In [ ]:
display(df.dropna(subset="SIFTcat").limit(50).toPandas())

それぞれのカラムでNAであるデータがいくつあるかをカウントしてみます。

In [ ]:
from pyspark.sql.functions import *
col_null_cnt_df =  df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns])

In [ ]:
display(col_null_cnt_df.toPandas())

反対に、NullやNaではなく値が入っている数を数えてみます。  
SIFTcat以外にもいくつかのカラムはそもそもデータが入っていないことがわかりました。

In [ ]:
col_not_null_cnt_df =  df.select([count(when(col(c).isNotNull(),c)).alias(c) for c in df.columns])
display(col_not_null_cnt_df.toPandas())

inferSchemaで取得されたSchemaを見てみます  
  
正確なところはやはり↓のような公式ドキュメントを見て決定していく必要がありそうです。  
https://cadd.gs.washington.edu/static/ReleaseNotes_CADD_v1.4.pdf

In [ ]:
df.printSchema()

### ディスクへの書き込み、読み込み
##### S3
データをdelta形式でs3へ保存します

In [ ]:
%%time
df.write.format("delta").mode("overwrite").save(delta_output_path)

s3から読み込む場合は、このようにします。

In [ ]:
df_s3 = spark.read.format("delta").load(delta_output_path)

### DataFrame操作
ここではRawScoreでsortしています。

In [ ]:
%%time
display(df_s3.sort("RawScore").select("#Chrom","Pos","Ref","Alt","Type","RawScore").limit(20).toPandas())

スコアの高い順番にsortする場合は次のようにdesc()で項目名を囲みます。

In [ ]:
%%time
display(df_s3.sort(desc("RawScore")).select("#Chrom","Pos","Ref","Alt","Type","RawScore").limit(20).toPandas())